In [1]:
import pandas as pd

In [2]:
tissue = 'Whole_Blood'

folder_to_model = '../gwas_component/eigengene_model/'
folder_to_1k_pred = '../processed_data/predixcan_1kgenome/'

df = pd.read_csv(folder_to_1k_pred + 'TW_%s_0.5.db_predicted_expression.txt.gz' % tissue,
                 sep = '\t', index_col = 1, )
df.columns = [g.split('.')[0] for g in df.columns]

df = df.drop('FID', axis = 1)

In [3]:
import pickle

with open(folder_to_model + '%s.p' % tissue, 'rb') as reader:
    pca_models = pickle.load(reader)
pca_models = [(k, pca_genes, pca_weight, sig_1k) for k, pca_genes, pca_weight, sig_1k in pca_models if k != 'grey']

In [4]:
import numpy as np
i_select = 80

k, pca_genes, pca_weight, sig_1k = pca_models[i_select]

In [5]:
df[pca_genes].head()

,ENSG00000240563,ENSG00000118729,ENSG00000143375,ENSG00000159516,ENSG00000159166,ENSG00000198626,ENSG00000078114,ENSG00000171798,ENSG00000134812,ENSG00000229183,...,ENSG00000089356,ENSG00000124466,ENSG00000142515,ENSG00000134317,ENSG00000185252,ENSG00000174564,ENSG00000069011,ENSG00000096696,ENSG00000148296,ENSG00000148297
IID,,,,,,,,,,,,,,,,,,,,,
HG00096,0.178384,0.158839,0.065014,0.026039,0.106295,0.239049,1.106435,-0.914735,0.314761,-0.414860,...,0.531028,-0.412548,-0.192133,-0.231996,0.370651,-0.066870,0.052659,-0.127172,-0.434646,-0.379292
HG00097,0.076622,0.071517,-0.083818,-0.347481,0.170927,0.412572,0.186642,-0.544955,-0.152159,-0.247089,...,0.427758,-0.095455,-0.011615,0.041611,0.311506,0.088463,0.158396,0.136383,0.292972,0.019885
HG00099,0.074277,0.293184,-0.004867,-0.002402,-0.198491,0.578420,-0.025849,-0.909177,0.255484,-0.226730,...,0.879668,-0.138168,-0.316569,-0.184708,0.411196,-0.066870,0.238432,0.038036,1.022826,0.634754
HG00100,-0.142766,0.337031,0.110451,0.114848,-0.072226,0.104031,-0.068626,-0.703975,-0.075389,-0.179441,...,0.760727,-0.109729,-0.325802,-0.263802,-0.003745,0.051532,0.115630,0.355889,-0.382653,-0.379292
HG00101,-0.060683,0.348695,0.229259,0.018520,0.002587,-0.166571,0.299824,-1.074977,0.144992,-0.250431,...,0.636215,-0.786160,-0.442318,0.012076,0.225680,-0.023835,0.000253,0.320911,0.292972,0.201428


In [6]:
mat = df[pca_genes].values

true_pheno = pca_weight.dot(mat.T) ## weights of PCs

In [7]:
import numpy as np

for p, s in zip([19.0, 49.0, 99.0, 199.0], ['05','02', '01', 'p5']):
    for r in range(1):
        tmp = true_pheno
        tmp = tmp + np.sqrt(p) * np.std(tmp) * np.random.randn(tmp.shape[0])

        pheno = pd.DataFrame(list(df.index))
        pheno.columns = ['FID']
        pheno['IID'] = list(df.index)
        pheno['phenotype'] = tmp
        pheno.to_csv('single_component/traits/single_component_%s_%02d.tsv' % (s, r), index = None, sep = '\t')

        #plt.plot(tmp,  w[0, :], '.')
        #plt.show()